## Metaclasses

Metaclass, the class of a class ([docs](https://docs.python.org/3/reference/datamodel.html#metaclasses))

Basic class creation:

> By default, classes are constructed using type(). The class body is executed in a new namespace and the class name is bound locally to the result of type(name, bases, namespace).

When a class definition is executed, the following steps occur:

* MRO entries are resolved;
* the appropriate metaclass is determined;
* the class namespace is prepared;
* the class body is executed;
* the class object is created.

Also:

“Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).”

When to use: 

* Avoiding decorators repetition or decorating all subclasses
    * if subclasses of a class use (the same) class decorator, can move that to a metaclass and pass that to the base class
* Validation of subclasses (working around dynamic typing, too)
    * attributes exist **and** values follow certain patterns 
* Registering subclasses
* A declarative way of building GUI
* Django ORM (e.g. adding attributes)

In [1]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

In [2]:
type(MySubclass) # __main__.Meta

__main__.Meta

In [3]:
a = MyClass()

> type is a metaclass, of which classes are instances

We can create a class with type, like so:


In [4]:
C = type("C", (), {})

In [5]:
type(C)

type

What happens at class creation?

In [6]:
class A:
    pass

a = A()

* A is called, it execute `__call__` of superclass, which is type
* parent's `__call__` invokes `__new__` and `__init__`
* a will be set to what `__new__` returns

So `type` is a class as well, so it has a `__new__` method.

In [7]:
type.__new__?

In [8]:
def new(cls):
    inst = object.__new__(cls)
    inst.attr = 1
    return inst
    

In [9]:
class A:
    pass


In [10]:
A.__new__ = new

In [11]:
a = A()

In [12]:
a.attr

1

However, we cannot alter `type` here.


In [13]:
type.__new__

<function type.__new__(*args, **kwargs)>

In [14]:
# type.__new__ = lambda cls: cls # TypeError: cannot set '__new__' attribute of immutable type 'type'

In order to customize class creation, we can use a metaclass.

In [19]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        k = super().__new__(cls, name, bases, dct)
        k.attr = 123
        return k

In [20]:
class A(metaclass=Meta):
    pass

In [21]:
type(A)

__main__.Meta

In [22]:
A.attr

123

In [23]:
A.__bases__

(object,)

A more complete example

In [24]:
class MyMeta(type):
    def __new__(cls, name, bases, dct):
        print(f"__new__ metaclass {cls} {name} {bases} {dct}")
        inst = super().__new__(cls, name, bases, dct)
        return inst

class A(metaclass=MyMeta):
    def __new__(cls, *args, **kwargs):
        print(f"__new__ {cls} {args} {kwargs}")
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self):
        print(f"__init__ {self}")
    

__new__ metaclass <class '__main__.MyMeta'> A () {'__module__': '__main__', '__qualname__': 'A', '__new__': <function A.__new__ at 0x7f5e867e5b80>, '__init__': <function A.__init__ at 0x7f5e86787040>, '__classcell__': <cell at 0x7f5e8677f280: empty>}


In [25]:
a = A()

__new__ <class '__main__.A'> () {}
__init__ <__main__.A object at 0x7f5e9c017f40>


In [26]:
a.x = 1

Example: A metaclass that disallows undocumented classes.

In [27]:
class DocsRequired(type):
    def __new__(cls, name, bases, dct):
        inst = super().__new__(cls, name, bases, dct)
        if not inst.__doc__:
            raise RuntimeError(f"class has no docs {cls}")
        return inst

In [29]:
class A(metaclass=DocsRequired):
    """
    Ok, with docs now.
    """
    pass

In [80]:
# class A(metaclass=DocsRequired): # RuntimeError: class has no docs <class '__main__.DocsRequired'>
#     pass

Some use cases for registration.

* registration, example: [task.py](https://github.com/spotify/luigi/blob/38b0c2b8d400f6fc57ebbdc4064aadd4f3d7b612/luigi/task.py#L147-L171), [task_register.py](https://github.com/spotify/luigi/blob/38b0c2b8d400f6fc57ebbdc4064aadd4f3d7b612/luigi/task_register.py#L39-L48)
* enforcing implementations of abstract methods, example [metrics.py](https://github.com/spotify/luigi/blob/38b0c2b8d400f6fc57ebbdc4064aadd4f3d7b612/luigi/metrics.py#L44-L73)

Abstract base classes, ABC provides a metaclass as well.

Module [abc]() provides a convenience layer as `abc.ABC` -- [abc.py](https://github.com/python/cpython/blob/91a8e002c21a5388c5152c5a4871b9a2d85f0fc1/Lib/abc.py#L184-L188)

Used in conjuntion with `abc.abstractmethod` class decorator to ensure all required methods are implemented.

> Requires that the metaclass is ABCMeta or derived from it.  A
    class that has a metaclass derived from ABCMeta cannot be
    instantiated unless all of its abstract methods are overridden.

### More examples

Another roundup of metaclasses.

In [53]:
class MetaKlass(type):
    
    def __new__(meta, name, bases, dct):
        # _new__ should be implemented when you want to control the creation of a new object (here: class),
        print(f"MetaKlass __new__ {meta} {name} {bases} {dct}")
        return super().__new__(meta, name, bases, dct)

    def __init__(cls, name, bases, dct):
        # __init__ should be implemented when you want to control the initialization of the new object after it has been created
        print(f"MetaKlass __init__ {cls} {name} {bases} {dct}")

In [54]:
m = MetaKlass("m", (), {})

MetaKlass __new__ <class '__main__.MetaKlass'> m () {}
MetaKlass __init__ <class '__main__.m'> m () {}


A metaclass new and init method are executed at class creation time. Also, if a metaclass is specified in a normal class declaration.

In [55]:
        
class Klass(metaclass=MetaKlass):
    
    def __new__(cls, *args, **kwargs):
        print(f"Klass __new__ {args} {kwargs}")
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print(f"Klass __init__ {args} {kwargs}")

MetaKlass __new__ <class '__main__.MetaKlass'> Klass () {'__module__': '__main__', '__qualname__': 'Klass', '__new__': <function Klass.__new__ at 0x7f5e8638a040>, '__init__': <function Klass.__init__ at 0x7f5e8638a670>, '__classcell__': <cell at 0x7f5ea00af550: empty>}
MetaKlass __init__ <class '__main__.Klass'> Klass () {'__module__': '__main__', '__qualname__': 'Klass', '__new__': <function Klass.__new__ at 0x7f5e8638a040>, '__init__': <function Klass.__init__ at 0x7f5e8638a670>, '__classcell__': <cell at 0x7f5ea00af550: MetaKlass object at 0x55927c8fc5d0>}


In [49]:
k = Klass()

Klass __new__ () {}
Klass __init__ () {}


In [50]:
g = Klass()

Klass __new__ () {}
Klass __init__ () {}


### Final Notes

* there is a few additional things, we haven't talked about like `__prepare__` executed before a metaclass' `__new__` method to prepare the namespace (https://peps.python.org/pep-3115/#invoking-the-metaclass) 

> `__prepare__` returns a dictionary-like object which is used to store the class member definitions during evaluation of the class body. In other words, the class body is evaluated as a function block (just like it is now), except that the local variables dictionary is replaced by the dictionary returned from `__prepare__`. This dictionary object can be a regular dictionary or a custom mapping type.

In [63]:
# The custom dictionary
class member_table(dict):
    def __init__(self):
        self.member_names = []

    def __setitem__(self, key, value):
        # if the key is not already defined, add to the
        # list of keys.
        if key not in self:
            self.member_names.append(key)

        # Call superclass
        dict.__setitem__(self, key, value)

# The metaclass
class OrderedClass(type):

    # The prepare function
    @classmethod
    def __prepare__(metacls, name, bases): # No keywords in this case
        return member_table()

    # The metaclass invocation
    def __new__(cls, name, bases, classdict):
        # Note that we replace the classdict with a regular
        # dict before passing it to the superclass, so that we
        # don't continue to record member names after the class
        # has been created.
        result = type.__new__(cls, name, bases, dict(classdict))
        result.member_names = classdict.member_names
        return result

class MyClass(metaclass=OrderedClass):
    # method1 goes in array element 0
    def method1(self):
        pass

    # method2 goes in array element 1
    def method2(self):
        pass


In [64]:
MyClass.member_names

['__module__', '__qualname__', 'method1', 'method2']

In [65]:
MyClass.__module__

'__main__'

In [66]:
MyClass.__qualname__

'MyClass'

In [67]:


class A(metaclass=OrderedClass):
    b = 2
    a = 1
    c = 3
    

In [68]:
a = A()

In [69]:
a.member_names

['__module__', '__qualname__', 'b', 'a', 'c']

### Example: A final class (that cannot be inherited from)

In [97]:
class Final(type):
    
    def __init__(cls, name, bases, classdict):
        for b in bases:
            if isinstance(b, Final):
                raise RuntimeError(f"cannot extend class: base {b} is final")
    

In [98]:
class A(metaclass=Final):
    pass

In [99]:
a = A()

In [100]:
type(a)

__main__.A

In [101]:
type(A)

__main__.Final

In [102]:
class B(A):
    pass

RuntimeError: cannot extend class: base <class '__main__.A'> is final

### Notes

* try "init" first, and when you need more, then use "new"